# Driver Overview

Let's run the WPS prep utility `ungrib` to transform a GFS 0-hour forecast GRIB files into initial conditions suitable for WRF or MPAS.

First `cd` into the `configs/` directory:

In [1]:
import os, shutil
os.chdir("../configs")
shutil.rmtree("driver-overview", ignore_errors=True)

Here's the config we'll use. Note the additional `platform:` and `ungrib.execution.batchargs:` blocks, which will be necessary for running on the batch system:

In [2]:
!cat do00.yaml

data:
  '{{ val.gfs }}': https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.{{ val.yyyymmdd }}/{{ val.hh }}/atmos/gfs.t{{ val.hh }}z.pgrb2.0p25.f000
platform:
  account: wrfruc
  scheduler: slurm
ungrib:
  execution:
    batchargs:
      cores: 1
      walltime: "00:01:00"
    executable: '{{ "CONDA_PREFIX" | env }}/bin/ungrib'
  gribfiles:
    - '{{ val.gfs }}'
  rundir: '{{ val.rundir }}'
  start: '{{ cycle }}'
  step: 1
  stop: '{{ cycle }}'
  vtable: '{{ "WPSFILES" | env }}/Vtable.GFS'
val:
  gfs: '{{ val.rundir }}/anl.grib2'
  hh: '{{ cycle.strftime("%H") }}'
  rundir: '{{ "PWD" | env }}/driver-overview'
  yyyymmdd: '{{ cycle.strftime("%Y%m%d") }}'


First, use `uw fs` to stage the GFS GRIB file from AWS:

In [3]:
!uw fs copy -c do00.yaml --key-path data --cycle 2025-09-21T06

[2025-09-22T19:51:15]     INFO Validating config against internal schema: files-to-stage
[2025-09-22T19:51:15]     INFO Schema validation succeeded for fs config
[2025-09-22T19:51:15]     INFO HTTP https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.20250921/06/atmos/gfs.t06z.pgrb2.0p25.f000 -> /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/anl.grib2: Executing
[2025-09-22T19:54:59]     INFO HTTP https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.20250921/06/atmos/gfs.t06z.pgrb2.0p25.f000 -> /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/anl.grib2: Ready
[2025-09-22T19:54:59]     INFO File copies: Ready


Now run `ungrib` via the UW driver, locally:

In [4]:
!uw ungrib run -c do00.yaml --cycle 2025-09-21T06

[2025-09-22T19:55:43]     INFO Validating config against internal schema: ungrib
[2025-09-22T19:55:43]     INFO Schema validation succeeded for ungrib config
[2025-09-22T19:55:43]     INFO Validating config against internal schema: platform
[2025-09-22T19:55:43]     INFO Schema validation succeeded for platform config
[2025-09-22T19:55:43]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/GRIBFILE.AAA: Executing
[2025-09-22T19:55:43]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/GRIBFILE.AAA: Ready
[2025-09-22T19:55:43]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/namelist.wps: Executing
[2025-09-22T19:55:43]     INFO Schema validation succeeded for user-updated config
[2025-09-22T19:55:43]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/namelist.wps: Ready
[2025-09-22T1

See the output:

In [5]:
!tree driver-overview

driver-overview
├── anl.grib2
├── FILE:2025-09-21_06
├── GRIBFILE.AAA -> /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/anl.grib2
├── namelist.wps
├── runscript.ungrib
├── runscript.ungrib.done
├── runscript.ungrib.out
├── ungrib.log
└── Vtable -> /tmp/pm/conda/envs/DEV-uwtools/etc/wps/Vtable.GFS

1 directory, 9 files


Note that an invocation again is a no-op:

In [6]:
!uw ungrib run -c do00.yaml --cycle 2025-09-21T06

[2025-09-22T19:57:17]     INFO Validating config against internal schema: ungrib
[2025-09-22T19:57:17]     INFO Schema validation succeeded for ungrib config
[2025-09-22T19:57:17]     INFO Validating config against internal schema: platform
[2025-09-22T19:57:17]     INFO Schema validation succeeded for platform config
[2025-09-22T19:57:17]     INFO 20250921 06Z ungrib run: Ready


The `ungrib` driver views its `runscript.ungrib.done` file as evidence of its readiness, so we'll need to remove it to force re-execution. We'll also remove `runscript.ungrib` so that it gets recreated with batch directives -- remember, drivers are idempotent, and `runscript.ungrib` already exists. For brevity, we'll just remove `runscript.ungrib*`.

In [7]:
%%bash
rm -fv driver-overview/runscript.ungrib*
uw ungrib run -c do00.yaml --cycle 2025-09-21T06 --batch
squeue -u $USER

removed 'driver-overview/runscript.ungrib'
removed 'driver-overview/runscript.ungrib.done'
removed 'driver-overview/runscript.ungrib.out'


[2025-09-22T20:03:08]     INFO Validating config against internal schema: ungrib
[2025-09-22T20:03:08]     INFO Schema validation succeeded for ungrib config
[2025-09-22T20:03:08]     INFO Validating config against internal schema: platform
[2025-09-22T20:03:08]     INFO Schema validation succeeded for platform config
[2025-09-22T20:03:08]     INFO 20250921 06Z ungrib runscript.ungrib: Executing
[2025-09-22T20:03:08]     INFO 20250921 06Z ungrib runscript.ungrib: Ready
[2025-09-22T20:03:08]     INFO 20250921 06Z ungrib provisioned run directory: Ready
[2025-09-22T20:03:08]     INFO 20250921 06Z ungrib run via batch submission: Executing
[2025-09-22T20:03:08]     INFO Running: sbatch /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/runscript.ungrib 2>&1 | tee /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/runscript.ungrib.submit in /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview
[2025-09-22T20:03:08]     INFO 202

     JOBID PARTITION  NAME                     USER             STATE        TIME TIME_LIMIT NODES NODELIST(REASON)
  16395458 hera       runscript.ungrib         Paul.Madden      PENDING      0:00       1:00     1 (None)


After a short time...

In [ ]:
%%bash
squeue -u $USER
tree driver-overview

Note the new batch directives in `runscript.ungrib`:

In [ ]:
!head driver-overview/runscript.ungrib

In [ ]:
!rm -rf driver-overview